In [27]:
import nltk
import pandas as pd
# import re
import string
import contractions
import os
from nltk import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords
from nltk.stem import  WordNetLemmatizer
import inflect
# os.chdir('/home/rinkita/courses/NLPSeminar/nlp-quora-kaggle')
# nltk.download('punkt')
# nltk.download('stopwords')
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from collections import defaultdict

import matplotlib.pyplot as plt

In [39]:
glove_word_embedding_path = "./data/embeddings/glove.840B.300d/glove.840B.300d.txt"
GLOVE_SIZE = 300
embedding_size=64
num_class=2


In [29]:
data = pd.read_csv("./data/train.csv")
x_train=data.question_text
max_document_len=x_train.map(lambda x: len(x.split(' '))).max()


In [30]:
# """Replace contractions in string of text"""
contracted_input = x_train.progress_apply(lambda row:contractions.fix(row))



  0%|          | 0/1306122 [00:00<?, ?it/s]

  0%|          | 271/1306122 [00:00<08:01, 2709.91it/s]

  0%|          | 814/1306122 [00:00<06:49, 3188.55it/s]

  0%|          | 1336/1306122 [00:00<06:01, 3609.94it/s]

  0%|          | 1818/1306122 [00:00<05:34, 3901.85it/s]

  0%|          | 2322/1306122 [00:00<05:11, 4183.64it/s]

  0%|          | 2859/1306122 [00:00<04:51, 4473.56it/s]

  0%|          | 3391/1306122 [00:00<04:37, 4696.33it/s]

  0%|          | 3896/1306122 [00:00<04:31, 4794.32it/s]

  0%|          | 4381/1306122 [00:00<04:30, 4805.51it/s]

  0%|          | 4862/1306122 [00:01<04:35, 4715.54it/s]

  0%|          | 5335/1306122 [00:01<04:36, 4696.25it/s]

  0%|          | 5862/1306122 [00:01<04:27, 4854.51it/s]

  0%|          | 6362/1306122 [00:01<04:25, 4896.53it/s]

  1%|          | 6853/1306122 [00:01<04:26, 4866.51it/s]

  1%|          | 7388/1306122 [00:01<04:19, 4999.87it/s]

  1%|          | 7890/1306122 [00:01<04:22, 4943.26it/s]

  1%|          | 8433/13061

 10%|█         | 131959/1306122 [00:25<03:45, 5210.39it/s]

 10%|█         | 132539/1306122 [00:25<03:38, 5367.96it/s]

 10%|█         | 133078/1306122 [00:25<03:41, 5299.78it/s]

 10%|█         | 133610/1306122 [00:26<03:42, 5280.06it/s]

 10%|█         | 134147/1306122 [00:26<03:40, 5303.76it/s]

 10%|█         | 134679/1306122 [00:26<03:44, 5220.65it/s]

 10%|█         | 135202/1306122 [00:26<03:46, 5173.13it/s]

 10%|█         | 135756/1306122 [00:26<03:42, 5267.59it/s]

 10%|█         | 136328/1306122 [00:26<03:36, 5392.90it/s]

 10%|█         | 136872/1306122 [00:26<03:36, 5405.39it/s]

 11%|█         | 137460/1306122 [00:26<03:30, 5538.93it/s]

 11%|█         | 138016/1306122 [00:26<03:41, 5267.92it/s]

 11%|█         | 138547/1306122 [00:26<03:43, 5212.74it/s]

 11%|█         | 139072/1306122 [00:27<03:45, 5180.61it/s]

 11%|█         | 139593/1306122 [00:27<03:45, 5166.62it/s]

 11%|█         | 140112/1306122 [00:27<03:50, 5063.39it/s]

 11%|█         | 140620/1306122 [00:27<0

 20%|██        | 263986/1306122 [00:51<03:24, 5107.49it/s]

 20%|██        | 264499/1306122 [00:51<03:26, 5050.82it/s]

 20%|██        | 265039/1306122 [00:51<03:22, 5150.68it/s]

 20%|██        | 265557/1306122 [00:51<03:21, 5159.28it/s]

 20%|██        | 266079/1306122 [00:51<03:20, 5175.21it/s]

 20%|██        | 266611/1306122 [00:51<03:19, 5217.34it/s]

 20%|██        | 267134/1306122 [00:51<03:21, 5161.95it/s]

 20%|██        | 267667/1306122 [00:51<03:19, 5208.72it/s]

 21%|██        | 268189/1306122 [00:51<03:22, 5113.79it/s]

 21%|██        | 268702/1306122 [00:52<03:22, 5113.95it/s]

 21%|██        | 269233/1306122 [00:52<03:20, 5170.65it/s]

 21%|██        | 269751/1306122 [00:52<03:21, 5142.11it/s]

 21%|██        | 270268/1306122 [00:52<03:21, 5148.10it/s]

 21%|██        | 270784/1306122 [00:52<03:21, 5146.50it/s]

 21%|██        | 271299/1306122 [00:52<03:22, 5122.34it/s]

 21%|██        | 271812/1306122 [00:52<03:23, 5071.84it/s]

 21%|██        | 272362/1306122 [00:52<0

 30%|███       | 395455/1306122 [01:16<02:56, 5165.54it/s]

 30%|███       | 395974/1306122 [01:16<03:03, 4954.07it/s]

 30%|███       | 396522/1306122 [01:16<02:58, 5099.43it/s]

 30%|███       | 397104/1306122 [01:16<02:51, 5294.25it/s]

 30%|███       | 397638/1306122 [01:16<02:52, 5257.13it/s]

 30%|███       | 398183/1306122 [01:17<02:50, 5311.75it/s]

 31%|███       | 398717/1306122 [01:17<02:51, 5290.38it/s]

 31%|███       | 399248/1306122 [01:17<02:52, 5271.56it/s]

 31%|███       | 399777/1306122 [01:17<02:53, 5224.07it/s]

 31%|███       | 400337/1306122 [01:17<02:50, 5325.62it/s]

 31%|███       | 400871/1306122 [01:17<02:50, 5317.34it/s]

 31%|███       | 401404/1306122 [01:17<02:51, 5287.88it/s]

 31%|███       | 401942/1306122 [01:17<02:50, 5314.65it/s]

 31%|███       | 402474/1306122 [01:17<02:52, 5242.72it/s]

 31%|███       | 403012/1306122 [01:17<02:50, 5282.53it/s]

 31%|███       | 403569/1306122 [01:18<02:48, 5362.53it/s]

 31%|███       | 404111/1306122 [01:18<0

 40%|████      | 527325/1306122 [01:41<02:47, 4642.11it/s]

 40%|████      | 527794/1306122 [01:42<02:52, 4517.53it/s]

 40%|████      | 528250/1306122 [01:42<02:54, 4447.38it/s]

 40%|████      | 528699/1306122 [01:42<02:54, 4452.21it/s]

 41%|████      | 529147/1306122 [01:42<02:55, 4426.62it/s]

 41%|████      | 529592/1306122 [01:42<02:55, 4416.43it/s]

 41%|████      | 530035/1306122 [01:42<02:57, 4373.12it/s]

 41%|████      | 530539/1306122 [01:42<02:50, 4552.02it/s]

 41%|████      | 531085/1306122 [01:42<02:41, 4790.74it/s]

 41%|████      | 531575/1306122 [01:42<02:40, 4822.34it/s]

 41%|████      | 532099/1306122 [01:42<02:36, 4938.98it/s]

 41%|████      | 532600/1306122 [01:43<02:35, 4960.05it/s]

 41%|████      | 533099/1306122 [01:43<02:38, 4885.17it/s]

 41%|████      | 533590/1306122 [01:43<02:39, 4845.17it/s]

 41%|████      | 534076/1306122 [01:43<02:40, 4809.33it/s]

 41%|████      | 534558/1306122 [01:43<02:42, 4736.81it/s]

 41%|████      | 535048/1306122 [01:43<0

...

 60%|██████    | 787922/1306122 [02:32<01:39, 5187.79it/s]

 60%|██████    | 788442/1306122 [02:32<01:40, 5151.67it/s]

 60%|██████    | 788979/1306122 [02:32<01:39, 5212.98it/s]

 60%|██████    | 789502/1306122 [02:33<01:40, 5147.41it/s]

 60%|██████    | 790046/1306122 [02:33<01:38, 5222.77it/s]

 61%|██████    | 790580/1306122 [02:33<01:38, 5255.90it/s]

 61%|██████    | 791107/1306122 [02:33<01:39, 5194.12it/s]

 61%|██████    | 791632/1306122 [02:33<01:38, 5210.21it/s]

 61%|██████    | 792154/1306122 [02:33<01:40, 5091.26it/s]

 61%|██████    | 792694/1306122 [02:33<01:39, 5177.62it/s]

 61%|██████    | 793213/1306122 [02:33<01:40, 5127.79it/s]

 61%|██████    | 793827/1306122 [02:33<01:34, 5394.19it/s]

 61%|██████    | 794371/1306122 [02:33<01:34, 5404.50it/s]

 61%|██████    | 794915/1306122 [02:34<01:34, 5411.12it/s]

 61%|██████    | 795459/1306122 [02:34<01:35, 5334.87it/s]

 61%|██████    | 795998/1306122 [02:34<01:35, 5350.69it/s]

 61%|██████    | 796544/1306122 [02:34<0

 70%|███████   | 918689/1306122 [02:58<01:16, 5072.67it/s]

 70%|███████   | 919208/1306122 [02:58<01:15, 5105.55it/s]

 70%|███████   | 919749/1306122 [02:58<01:14, 5192.24it/s]

 70%|███████   | 920269/1306122 [02:58<01:14, 5183.76it/s]

 70%|███████   | 920796/1306122 [02:58<01:13, 5208.03it/s]

 71%|███████   | 921336/1306122 [02:58<01:13, 5256.88it/s]

 71%|███████   | 921862/1306122 [02:58<01:13, 5222.71it/s]

 71%|███████   | 922385/1306122 [02:58<01:13, 5206.12it/s]

 71%|███████   | 922906/1306122 [02:59<01:13, 5185.98it/s]

 71%|███████   | 923461/1306122 [02:59<01:12, 5289.95it/s]

 71%|███████   | 924052/1306122 [02:59<01:09, 5461.55it/s]

 71%|███████   | 924606/1306122 [02:59<01:09, 5483.14it/s]

 71%|███████   | 925162/1306122 [02:59<01:09, 5504.18it/s]

 71%|███████   | 925714/1306122 [02:59<01:10, 5393.04it/s]

 71%|███████   | 926255/1306122 [02:59<01:11, 5336.82it/s]

 71%|███████   | 926790/1306122 [02:59<01:12, 5263.31it/s]

 71%|███████   | 927318/1306122 [02:59<0

 80%|████████  | 1049214/1306122 [03:23<00:49, 5149.54it/s]

 80%|████████  | 1049730/1306122 [03:23<00:50, 5037.09it/s]

 80%|████████  | 1050275/1306122 [03:23<00:49, 5153.90it/s]

 80%|████████  | 1050792/1306122 [03:23<00:49, 5153.46it/s]

 80%|████████  | 1051333/1306122 [03:23<00:48, 5224.82it/s]

 81%|████████  | 1051904/1306122 [03:23<00:47, 5361.37it/s]

 81%|████████  | 1052442/1306122 [03:23<00:47, 5363.75it/s]

 81%|████████  | 1052999/1306122 [03:24<00:46, 5423.94it/s]

 81%|████████  | 1053543/1306122 [03:24<00:47, 5367.29it/s]

 81%|████████  | 1054081/1306122 [03:24<00:47, 5298.19it/s]

 81%|████████  | 1054612/1306122 [03:24<00:47, 5257.52it/s]

 81%|████████  | 1055195/1306122 [03:24<00:46, 5415.02it/s]

 81%|████████  | 1055739/1306122 [03:24<00:46, 5417.19it/s]

 81%|████████  | 1056282/1306122 [03:24<00:47, 5312.81it/s]

 81%|████████  | 1056815/1306122 [03:24<00:47, 5242.78it/s]

 81%|████████  | 1057352/1306122 [03:24<00:47, 5279.10it/s]

 81%|████████  | 1057889

 90%|█████████ | 1175962/1306122 [03:48<00:25, 5158.65it/s]

 90%|█████████ | 1176479/1306122 [03:48<00:25, 5056.20it/s]

 90%|█████████ | 1177011/1306122 [03:48<00:25, 5129.91it/s]

 90%|█████████ | 1177540/1306122 [03:48<00:24, 5176.69it/s]

 90%|█████████ | 1178095/1306122 [03:48<00:24, 5282.12it/s]

 90%|█████████ | 1178625/1306122 [03:48<00:24, 5280.27it/s]

 90%|█████████ | 1179154/1306122 [03:48<00:24, 5243.97it/s]

 90%|█████████ | 1179699/1306122 [03:49<00:23, 5300.41it/s]

 90%|█████████ | 1180239/1306122 [03:49<00:23, 5328.80it/s]

 90%|█████████ | 1180773/1306122 [03:49<00:23, 5302.77it/s]

 90%|█████████ | 1181308/1306122 [03:49<00:23, 5316.73it/s]

 90%|█████████ | 1181840/1306122 [03:49<00:23, 5248.50it/s]

 91%|█████████ | 1182366/1306122 [03:49<00:24, 5126.95it/s]

 91%|█████████ | 1182884/1306122 [03:49<00:23, 5140.76it/s]

 91%|█████████ | 1183399/1306122 [03:49<00:24, 5090.88it/s]

 91%|█████████ | 1183909/1306122 [03:49<00:24, 5059.97it/s]

 91%|█████████ | 1184416

100%|█████████▉| 1305609/1306122 [04:13<00:00, 5246.43it/s]

100%|██████████| 1306122/1306122 [04:13<00:00, 5153.30it/s]

In [31]:
def clean_text(question_text):
    question_text = question_text.translate(str.maketrans('','',string.punctuation))
    question_text = question_text.translate(str.maketrans('','','1234567890'))
    question_text = " ".join(question_text.split())    
    question_text = question_text.split()
    # stop_words = set(stopwords.words('english'))
    # question_text = [w for w in question_text if not w in stop_words and len(w) >= 3]
    question_text = [w for w in question_text if len(w) >= 3]
    question_text = [w.strip('/"“”') for w in question_text ]
    question_text = " ".join(question_text)
    return question_text

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def lower_case(words):
    """Turn words to lower case"""
    lowered = []
    for word in words:
        lowered.append(word.lower())
    return lowered

In [32]:


def extract_features(question_texts):  
    word2index = {}
    word2count = {}
    index2word = {0: "pad"}
    n_words = 1  # Count SOS and EOS
    for text in question_texts:
        for word in text:             
            if word not in word2index:
                word2index[word] = n_words
                word2count[word] = 1
                index2word[n_words] = word
                n_words += 1
            else:
                word2count[word] += 1
    return word2index,word2count,index2word,n_words

    

#make all the input text of the same size as size max_length input senetnce, padding with word "PAD"(zero padding)
def make_input(sentence):
    sent_len = len(sentence.split(' '))
    if sent_len < max_document_len:
        padded_sentence = sentence + (max_document_len - len(sentence.split(' '))) * " PAD"
    else:
        padded_sentence = sentence

    return padded_sentence,sent_len


#function to return batch of data
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens,input_metadata):
    
    #shuffling and creating training batch data of batch_size
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    
    #converting sentence to index vector using word2index dictionary
    x = [[input_metadata.word2index[word] for word in data_x[i].lower().split(' ')]for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens



In [33]:
cleaned_text = contracted_input.progress_apply(lambda row:clean_text(row))

word_tokenised = cleaned_text.progress_apply(lambda text:nltk.word_tokenize(text))

lemmatized_words = word_tokenised.progress_apply(lambda word_list:lemmatize_verbs(word_list))

#lower_cased_words = word_tokenised.progress_apply(lambda word_list:lower_case(lemmatized_words))
 



  0%|          | 0/1306122 [00:00<?, ?it/s]

  0%|          | 3061/1306122 [00:00<00:42, 30606.89it/s]

  1%|          | 7934/1306122 [00:00<00:37, 34450.14it/s]

  1%|          | 13211/1306122 [00:00<00:33, 38454.53it/s]

  1%|▏         | 18292/1306122 [00:00<00:31, 41479.84it/s]

  2%|▏         | 23676/1306122 [00:00<00:28, 44537.21it/s]

  2%|▏         | 29181/1306122 [00:00<00:27, 47243.12it/s]

  3%|▎         | 34840/1306122 [00:00<00:25, 49704.65it/s]

  3%|▎         | 40023/1306122 [00:00<00:25, 50322.48it/s]

  4%|▎         | 45747/1306122 [00:00<00:24, 52215.45it/s]

  4%|▍         | 51391/1306122 [00:01<00:23, 53412.66it/s]

  4%|▍         | 56729/1306122 [00:01<00:23, 52930.05it/s]

  5%|▍         | 62021/1306122 [00:01<00:23, 52513.19it/s]

  5%|▌         | 67273/1306122 [00:01<00:23, 51828.10it/s]

  6%|▌         | 72779/1306122 [00:01<00:23, 52755.76it/s]

  6%|▌         | 78486/1306122 [00:01<00:22, 53979.09it/s]

  6%|▋         | 83895/1306122 [00:01<00:22, 53501.81it

 98%|█████████▊| 1278206/1306122 [00:25<00:00, 51697.04it/s]

 98%|█████████▊| 1283397/1306122 [00:25<00:00, 49022.14it/s]

 99%|█████████▊| 1288593/1306122 [00:25<00:00, 49867.04it/s]

 99%|█████████▉| 1293969/1306122 [00:25<00:00, 50973.42it/s]

100%|█████████▉| 1299612/1306122 [00:25<00:00, 52494.83it/s]

100%|█████████▉| 1304962/1306122 [00:25<00:00, 52792.07it/s]

100%|██████████| 1306122/1306122 [00:25<00:00, 50250.38it/s]

  0%|          | 0/1306122 [00:00<?, ?it/s]

  0%|          | 294/1306122 [00:00<07:24, 2936.50it/s]

  0%|          | 746/1306122 [00:00<06:37, 3280.13it/s]

  0%|          | 1196/1306122 [00:00<06:05, 3569.99it/s]

  0%|          | 1651/1306122 [00:00<05:41, 3816.31it/s]

  0%|          | 2102/1306122 [00:00<05:26, 3999.14it/s]

  0%|          | 2535/1306122 [00:00<05:18, 4092.24it/s]

  0%|          | 2985/1306122 [00:00<05:09, 4206.14it/s]

  0%|          | 3477/1306122 [00:00<04:56, 4396.95it/s]

  0%|          | 3908/1306122 [00:00<04:58, 4356.20it/s]

 

  9%|▊         | 113781/1306122 [00:24<04:29, 4416.16it/s]

  9%|▊         | 114225/1306122 [00:25<04:29, 4418.14it/s]

  9%|▉         | 114678/1306122 [00:25<04:27, 4449.69it/s]

  9%|▉         | 115143/1306122 [00:25<04:24, 4507.91it/s]

  9%|▉         | 115621/1306122 [00:25<04:19, 4585.59it/s]

  9%|▉         | 116081/1306122 [00:25<04:29, 4415.39it/s]

  9%|▉         | 116602/1306122 [00:25<04:17, 4626.40it/s]

  9%|▉         | 117069/1306122 [00:25<04:16, 4637.04it/s]

  9%|▉         | 117571/1306122 [00:25<04:10, 4742.94it/s]

  9%|▉         | 118048/1306122 [00:25<04:12, 4706.41it/s]

  9%|▉         | 118521/1306122 [00:25<04:14, 4673.57it/s]

  9%|▉         | 118990/1306122 [00:26<04:14, 4673.01it/s]

  9%|▉         | 119466/1306122 [00:26<04:12, 4696.56it/s]

  9%|▉         | 119956/1306122 [00:26<04:09, 4754.99it/s]

  9%|▉         | 120433/1306122 [00:26<04:11, 4709.23it/s]

  9%|▉         | 120920/1306122 [00:26<04:09, 4751.90it/s]

  9%|▉         | 121396/1306122 [00:26<0

 18%|█▊        | 230043/1306122 [00:50<03:48, 4710.11it/s]

 18%|█▊        | 230516/1306122 [00:50<03:50, 4668.21it/s]

 18%|█▊        | 230986/1306122 [00:50<03:49, 4677.23it/s]

 18%|█▊        | 231455/1306122 [00:50<03:54, 4585.03it/s]

 18%|█▊        | 231915/1306122 [00:50<03:56, 4545.86it/s]

 18%|█▊        | 232371/1306122 [00:50<03:58, 4506.19it/s]

 18%|█▊        | 232848/1306122 [00:51<03:54, 4581.29it/s]

 18%|█▊        | 233307/1306122 [00:51<03:58, 4492.52it/s]

 18%|█▊        | 233758/1306122 [00:51<04:13, 4234.62it/s]

 18%|█▊        | 234186/1306122 [00:51<04:18, 4154.31it/s]

 18%|█▊        | 234643/1306122 [00:51<04:11, 4268.22it/s]

 18%|█▊        | 235080/1306122 [00:51<04:09, 4297.13it/s]

 18%|█▊        | 235512/1306122 [00:51<04:16, 4169.08it/s]

 18%|█▊        | 235965/1306122 [00:51<04:10, 4270.78it/s]

 18%|█▊        | 236395/1306122 [00:51<04:12, 4241.87it/s]

 18%|█▊        | 236851/1306122 [00:52<04:06, 4332.32it/s]

 18%|█▊        | 237286/1306122 [00:52<0

 26%|██▋       | 344309/1306122 [01:19<03:42, 4317.00it/s]

 26%|██▋       | 344808/1306122 [01:19<03:33, 4498.70it/s]

 26%|██▋       | 345304/1306122 [01:19<03:27, 4627.45it/s]

 26%|██▋       | 345806/1306122 [01:19<03:22, 4737.99it/s]

 27%|██▋       | 346286/1306122 [01:19<03:24, 4696.16it/s]

 27%|██▋       | 346760/1306122 [01:19<03:29, 4572.93it/s]

 27%|██▋       | 347230/1306122 [01:19<03:28, 4607.54it/s]

 27%|██▋       | 347709/1306122 [01:19<03:25, 4659.15it/s]

 27%|██▋       | 348177/1306122 [01:20<03:25, 4664.03it/s]

 27%|██▋       | 348666/1306122 [01:20<03:22, 4726.97it/s]

 27%|██▋       | 349144/1306122 [01:20<03:21, 4742.16it/s]

 27%|██▋       | 349621/1306122 [01:20<03:21, 4749.66it/s]

 27%|██▋       | 350097/1306122 [01:20<03:21, 4747.37it/s]

 27%|██▋       | 350573/1306122 [01:20<03:21, 4748.85it/s]

 27%|██▋       | 351049/1306122 [01:20<03:21, 4735.29it/s]

 27%|██▋       | 351534/1306122 [01:20<03:20, 4767.83it/s]

 27%|██▋       | 352011/1306122 [01:20<0

...

 89%|████████▉ | 1167656/1306122 [04:15<00:30, 4605.25it/s]

 89%|████████▉ | 1168147/1306122 [04:16<00:29, 4691.88it/s]

 89%|████████▉ | 1168618/1306122 [04:16<00:29, 4687.25it/s]

 90%|████████▉ | 1169088/1306122 [04:16<00:29, 4637.63it/s]

 90%|████████▉ | 1169553/1306122 [04:16<00:29, 4579.59it/s]

 90%|████████▉ | 1170037/1306122 [04:16<00:29, 4654.11it/s]

 90%|████████▉ | 1170504/1306122 [04:16<00:29, 4641.45it/s]

 90%|████████▉ | 1170969/1306122 [04:16<00:29, 4623.23it/s]

 90%|████████▉ | 1171461/1306122 [04:16<00:28, 4707.46it/s]

 90%|████████▉ | 1171936/1306122 [04:16<00:28, 4718.92it/s]

 90%|████████▉ | 1172419/1306122 [04:16<00:28, 4750.10it/s]

 90%|████████▉ | 1172895/1306122 [04:17<00:28, 4744.21it/s]

 90%|████████▉ | 1173370/1306122 [04:17<00:28, 4682.53it/s]

 90%|████████▉ | 1173841/1306122 [04:17<00:28, 4690.51it/s]

 90%|████████▉ | 1174311/1306122 [04:17<00:28, 4640.08it/s]

 90%|████████▉ | 1174776/1306122 [04:17<00:28, 4633.79it/s]

 90%|████████▉ | 1175240

 98%|█████████▊| 1283943/1306122 [04:40<00:04, 4613.58it/s]

 98%|█████████▊| 1284419/1306122 [04:40<00:04, 4655.05it/s]

 98%|█████████▊| 1284886/1306122 [04:41<00:04, 4629.67it/s]

 98%|█████████▊| 1285381/1306122 [04:41<00:04, 4720.77it/s]

 98%|█████████▊| 1285854/1306122 [04:41<00:04, 4712.80it/s]

 98%|█████████▊| 1286326/1306122 [04:41<00:04, 4635.13it/s]

 99%|█████████▊| 1286791/1306122 [04:41<00:04, 4511.27it/s]

 99%|█████████▊| 1287258/1306122 [04:41<00:04, 4557.48it/s]

 99%|█████████▊| 1287720/1306122 [04:41<00:04, 4574.61it/s]

 99%|█████████▊| 1288204/1306122 [04:41<00:03, 4649.12it/s]

 99%|█████████▊| 1288700/1306122 [04:41<00:03, 4738.14it/s]

 99%|█████████▊| 1289178/1306122 [04:41<00:03, 4748.71it/s]

 99%|█████████▊| 1289654/1306122 [04:42<00:03, 4740.36it/s]

 99%|█████████▉| 1290129/1306122 [04:42<00:03, 4675.42it/s]

 99%|█████████▉| 1290601/1306122 [04:42<00:03, 4683.96it/s]

 99%|█████████▉| 1291070/1306122 [04:42<00:03, 4631.48it/s]

 99%|█████████▉| 1291534

 22%|██▏       | 287352/1306122 [00:19<01:08, 14882.81it/s]

 22%|██▏       | 288846/1306122 [00:20<01:10, 14500.99it/s]

 22%|██▏       | 290303/1306122 [00:20<01:10, 14487.79it/s]

 22%|██▏       | 291930/1306122 [00:20<01:07, 14979.32it/s]

 22%|██▏       | 293517/1306122 [00:20<01:06, 15234.98it/s]

 23%|██▎       | 295047/1306122 [00:20<01:07, 14950.10it/s]

 23%|██▎       | 296592/1306122 [00:20<01:06, 15095.45it/s]

 23%|██▎       | 298106/1306122 [00:20<01:07, 14986.89it/s]

 23%|██▎       | 299834/1306122 [00:20<01:04, 15607.11it/s]

 23%|██▎       | 301404/1306122 [00:20<01:05, 15384.72it/s]

 23%|██▎       | 302971/1306122 [00:20<01:04, 15466.91it/s]

 23%|██▎       | 304523/1306122 [00:21<01:05, 15273.31it/s]

 23%|██▎       | 306055/1306122 [00:21<01:06, 15112.45it/s]

 24%|██▎       | 307570/1306122 [00:21<01:07, 14873.83it/s]

 24%|██▎       | 309090/1306122 [00:21<01:06, 14968.24it/s]

 24%|██▍       | 310697/1306122 [00:21<01:05, 15279.90it/s]

 24%|██▍       | 312229/

 50%|█████     | 656758/1306122 [00:46<00:47, 13810.99it/s]

 50%|█████     | 658244/1306122 [00:46<00:45, 14108.58it/s]

 51%|█████     | 659692/1306122 [00:46<00:45, 14216.38it/s]

 51%|█████     | 661355/1306122 [00:46<00:43, 14862.56it/s]

 51%|█████     | 662877/1306122 [00:46<00:42, 14966.60it/s]

 51%|█████     | 664383/1306122 [00:46<00:42, 14957.76it/s]

 51%|█████     | 665939/1306122 [00:46<00:42, 15131.11it/s]

 51%|█████     | 667517/1306122 [00:46<00:41, 15318.54it/s]

 51%|█████     | 669053/1306122 [00:47<00:42, 15166.80it/s]

 51%|█████▏    | 670639/1306122 [00:47<00:41, 15366.00it/s]

 51%|█████▏    | 672179/1306122 [00:47<00:41, 15300.78it/s]

 52%|█████▏    | 673712/1306122 [00:47<00:41, 15281.32it/s]

 52%|█████▏    | 675242/1306122 [00:47<00:41, 15142.15it/s]

 52%|█████▏    | 676758/1306122 [00:47<00:42, 14948.37it/s]

 52%|█████▏    | 678317/1306122 [00:47<00:41, 15134.42it/s]

 52%|█████▏    | 679833/1306122 [00:47<00:41, 14947.47it/s]

 52%|█████▏    | 681335/

 79%|███████▊  | 1025692/1306122 [01:11<00:20, 13894.11it/s]

 79%|███████▊  | 1027176/1306122 [01:11<00:19, 14162.71it/s]

 79%|███████▉  | 1028738/1306122 [01:11<00:19, 14570.47it/s]

 79%|███████▉  | 1030201/1306122 [01:11<00:19, 14502.91it/s]

 79%|███████▉  | 1031765/1306122 [01:11<00:18, 14825.64it/s]

 79%|███████▉  | 1033252/1306122 [01:11<00:19, 13972.12it/s]

 79%|███████▉  | 1034663/1306122 [01:11<00:20, 13084.06it/s]

 79%|███████▉  | 1035993/1306122 [01:11<00:21, 12497.53it/s]

 79%|███████▉  | 1037328/1306122 [01:12<00:21, 12739.31it/s]

 80%|███████▉  | 1038665/1306122 [01:12<00:20, 12917.72it/s]

 80%|███████▉  | 1039968/1306122 [01:12<00:20, 12782.34it/s]

 80%|███████▉  | 1041511/1306122 [01:12<00:19, 13475.07it/s]

 80%|███████▉  | 1042875/1306122 [01:12<00:19, 13167.83it/s]

 80%|███████▉  | 1044248/1306122 [01:12<00:19, 13331.10it/s]

 80%|████████  | 1045651/1306122 [01:12<00:19, 13529.84it/s]

 80%|████████  | 1047081/1306122 [01:12<00:18, 13745.21it/s]

 80%|███

In [40]:
word2index,word2count,index2word,n_words = extract_features(lemmatized_words)

In [41]:
def get_glove(path_to_glove,word2index_map):
    embedding_weights = {}
    count_all_words = 0
    with open(path_to_glove,'r') as f:  
        for line in f:
            vals = line.split(' ')
            word = str(vals[0])
            if word in word2index_map:                
                count_all_words += 1                                 
                coefs = np.asarray(vals[1:],dtype='float32')
                coefs /= np.linalg.norm(coefs)
                embedding_weights[word] = coefs
    # _WORD_NOT_FOUND = [0.0]* GLOVE_SIZE
    # embedding_weights = defaultdict(lambda: _WORD_NOT_FOUND)
    return embedding_weights

In [42]:

word2embedding_dict = get_glove(glove_word_embedding_path,word2index)


In [43]:
embedding_matrix = np.zeros((n_words,GLOVE_SIZE))

out_of_vocabulary = {}

embeddig_available = 0
for word,index  in word2index.items():
    try:        
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
    except:
        out_of_vocabulary[word] = word2index[word]
        embedding_matrix[index,:] = [0.0]* GLOVE_SIZE
       



In [44]:
print("vocabulary size is {}".format(n_words))
print("no. of words for which no pre-trained embedding found is {}".format(len(out_of_vocabulary)))
print("no. of words for which pre-trained embedding is available{}".format(n_words-len(out_of_vocabulary)))
print("embedding found for {} percentage of vocab".format(((n_words-len(out_of_vocabulary))/n_words)*100))
# out_of_vocabulary



vocabulary size is 259595
no. of words for which no pre-trained embedding found is 100244
no. of words for which pre-trained embedding is available159351
embedding found for 61.384464261638314 percentage of vocab


In [46]:
#make all the input text of the same size as size max_length input senetnce, padding with word "PAD"(zero padding)
def make_input(sentence):
    sent_len = len(sentence.split(' '))
    if sent_len < max_document_len:
        padded_sentence = sentence + (max_document_len - len(sentence.split(' '))) * " PAD"
    else:
        padded_sentence = sentence

    return padded_sentence,sent_len


#function to return batch of data
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens,input_metadata):
    
    #shuffling and creating training batch data of batch_size
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    
    #converting sentence to index vector using word2index dictionary
    x = [[input_metadata.word2index[word] for word in data_x[i].lower().split(' ')]for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens